In [1]:
from datasets import load_dataset

articles = load_dataset('online_news_popularity_data')

Reusing dataset online_news_popularity_data (/home/azureuser/.cache/huggingface/datasets/online_news_popularity_data/online_news_popularity_data/1.0.0/f3e03630a13ebe013884d6a83c7ec52cb4eec2c0f6012f710c9dba58aa719fcd)


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
articles.num_rows

{'train': 31686, 'validation': 7922}

In [2]:
from random import shuffle

def duplicate_shuffle_concatenate_texts(batch):
    title = list(batch['title'])
    content = list(batch['content'])
    full_text = [t + ' \n\n' + c for t,c in zip(title, content)]
    texts = ''
    for _ in range(3):
        shuffle(full_text)
        texts += ' \n\n\n'.join(full_text)
        texts += ' \n\n\n'
    texts = texts[:-3] 
    result = {'text': []}
    for i in range(len(texts)//800):
        result['text'].append(texts[i*800: (i+1)*800])
    return result

In [3]:
article_texts = articles.shuffle().map(duplicate_shuffle_concatenate_texts, batched = True,
            batch_size = 64, remove_columns=articles["train"].column_names)

Parameter 'function'=<function duplicate_shuffle_concatenate_texts at 0x7fae4c048670> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/496 [00:00<?, ?ba/s]

  0%|          | 0/124 [00:00<?, ?ba/s]

In [8]:
article_texts.num_rows

{'train': 300997, 'validation': 75460}

In [9]:
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [10]:
def preprocess_function(batch):
    return tokenizer(batch['text'], padding = False, truncation = True)

In [11]:
tokenized_article_texts = article_texts.shuffle().map(
    preprocess_function,
    batched=True,
    num_proc=12,
    batch_size = 128,
    remove_columns= ['text'],
)

#0:   0%|          | 0/196 [00:00<?, ?ba/s]

#1:   0%|          | 0/196 [00:00<?, ?ba/s]

#2:   0%|          | 0/196 [00:00<?, ?ba/s]

#3:   0%|          | 0/196 [00:00<?, ?ba/s]

#4:   0%|          | 0/196 [00:00<?, ?ba/s]

...

#8:   0%|          | 0/50 [00:00<?, ?ba/s]

#6:   0%|          | 0/50 [00:00<?, ?ba/s]

#9:   0%|          | 0/50 [00:00<?, ?ba/s]

#10:   0%|          | 0/50 [00:00<?, ?ba/s]

#11:   0%|          | 0/50 [00:00<?, ?ba/s]

In [12]:
tokenized_article_texts

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 300997
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 75460
    })
})

In [13]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [21]:
from transformers import AutoModelForMaskedLM, DistilBertForMaskedLM

model_ckpt = 'domain_adaptation_final'
model = DistilBertForMaskedLM.from_pretrained('domain_adaptation_final')

In [15]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [22]:
from transformers import TrainingArguments, Trainer

batch_size = 8

training_args = TrainingArguments(
                output_dir="additional_domain_adaptation",
                evaluation_strategy="epoch",
                learning_rate=1e-4, #recommended
                per_device_train_batch_size = batch_size,
                per_device_eval_batch_size = batch_size,
                num_train_epochs=4,
                weight_decay=0.01,
                push_to_hub=False,
                disable_tqdm = False,
                fp16 = True,
                resume_from_checkpoint = False)

trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=tokenized_article_texts["train"],
    eval_dataset=tokenized_article_texts["validation"].shuffle().select(range(800)),
    data_collator=data_collator,
)

trainer.train()

Using amp half precision backend
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 300997
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 150500


Epoch,Training Loss,Validation Loss


Attempted to log scalar metric loss:
2.2147
Attempted to log scalar metric learning_rate:
9.966777408637874e-05
Attempted to log scalar metric epoch:
0.01
Attempted to log scalar metric loss:
2.31
Attempted to log scalar metric learning_rate:
9.933554817275748e-05
Attempted to log scalar metric epoch:
0.03


In [ ]:
trainer.save_model('domain_adaptation_final_2')